# European Centre for Disease Prevention and Control Dataset

In [ ]:
import pandas as pd
import datetime
import pycountry
import re
import os
import numpy as np

In [ ]:
# papermill parameters
output_folder = "../output/"

### Fetch data

In [ ]:
df = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv")

### Parse date

In [ ]:
df["dateRep"] = pd.to_datetime(df["dateRep"], format="%d/%m/%Y")

### Add difference

In [ ]:
df['CASES_SINCE_PREV_WEEK'] = df.groupby(['countriesAndTerritories','continentExp'])['cases'].diff().fillna(0).astype(int)
df['DEATHS_SINCE_PREV_WEEK'] = df.groupby(['countriesAndTerritories','continentExp'])['deaths'].diff().fillna(0).astype(int)

### Drop cols

In [ ]:
df = df.drop(columns=["year", "countryterritoryCode"])

In [ ]:
int_conveyance = df["geoId"].loc["JPG11668" == df["geoId"]].index
df["geoId"].iloc[int_conveyance] = np.nan
df["popData2019"].iloc[int_conveyance] = np.nan
df["continentExp"].iloc[int_conveyance] = np.nan
df["countriesAndTerritories"].iloc[int_conveyance] = "Cases on an international conveyance Japan"

### Resolve Country/Region name

In [ ]:
country_codes = df["geoId"].unique()
for code in country_codes:
    try:
        pyc = pycountry.countries.get(alpha_2=code)
        if pyc:
            df["countriesAndTerritories"].loc[code == df["geoId"]] = pyc.name
    except LookupError:
        df["countriesAndTerritories"].loc[code == df["geoId"]] = None

### Set Last Update Date and Last Reported Flag

In [ ]:
df["LAST_UPDATE_DATE"] = datetime.datetime.utcnow()
df["LAST_REPORTED_FLAG"] = df["dateRep"].max() == df["dateRep"]

### Rename Cols

In [ ]:
df = df.rename(columns={
    "dateRep": "DATE", 
    "countriesAndTerritories": "COUNTRY_REGION", 
    "geoId": "ISO3166_1", 
    "popData2018": "POPULATION",
    "cases": "cases_weekly",
    "deaths": "deaths_weekly"
})

### Save dataframe

In [ ]:
df.to_csv(output_folder + "ECDC_GLOBAL_WEEKLY.csv", index=False, columns=[
    "COUNTRY_REGION",
    "continentExp",
    "ISO3166_1",
    "cases_weekly",
    "deaths_weekly",
    "CASES_SINCE_PREV_WEEK",
    "DEATHS_SINCE_PREV_WEEK",
    "popData2019",
    "DATE",
    "LAST_UPDATE_DATE",
    "LAST_REPORTED_FLAG"
])